<a href="https://colab.research.google.com/github/RxnAch/ProjectsOnDeepLearning/blob/main/Chest_X_Ray_Images(Pneumonia).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [1]:
%matplotlib inline
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import seaborn as sns
import skimage
from skimage import io, transform 
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets,models,transforms

#Load Dataset 
- test set
- train set
- validation set

In [2]:
data_dir = "/content/drive/MyDrive/datasets/kaggle_dataset/ChestXray/chest_xray/chest_xray"
TEST = 'test'
TRAIN = 'train'
VAL = 'val'

#Data Preprocessing and Augmentation

Deeplearning models requires a lot of data.In fact,the more the data,the better the performance of the model.

Image Augmentation is the process of generating new images for training our deep learning models. These images are generated using the existing training images and hence we don't have to collect them manually.

**transforms.Compose** just clubs all the transforms provided to it. So, all the transforms in the transforms.Compose are applied to the input one by one.

#**Train transforms**
**transforms.RandomResizedCrop(224)**:

 This will extract a patch of size (224, 224) from your input image randomly. So, it might pick this path from topleft, bottomright or anywhere in between. So, you are doing data augmentation in this part. Also, changing this value won't play nice with the fully-connected layers in your model, so not advised to change this.

**transforms.RandomHorizontalFlip():** 

Once we have our image of size (224, 224), we can choose to flip it. This is another part of data augmentation.
transforms.ToTensor(): This just converts your input image to PyTorch tensor.


**transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]):** This is just input data scaling and these values (mean and std) must have been precomputed for your dataset. Changing these values is also not advised.

#**Validation transforms**
**transforms.Resize(256):**

 First your input image is resized to be of size (256, 256)
**transforms.CentreCrop(224):** 

Crops the center part of the image of shape (224, 224)

In [ ]:
def data_transforms(phase):
  if phase == TRAIN:
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
  if phase == VAL:
    transform = transforms.Compose([
         transforms.Resize(256),
         transforms.CenterCrop(224),
         transforms.ToTensor(),
         transforms.
    ])